In [1]:
# Import necessary libraries
import pandas as pd # For data manipulation and analysis
import numpy as np # For numerical operations
import pickle # For serializing and deserializing Python objects
from modelling_utils import train_knn_classifier, evaluate_model, plot_model_comparison, extract_model_features, compute_knn_indices, compute_overlap_matrix, plot_overlap_matrix, split_shuffle_data

In [5]:
# Load features from a pickle file
feature_dict_path = "/home/suraj/Repositories/FM-extractors-radiomics/evaluation/features/nsclc_radiomics.pkl" # Path to the pickle file containing features
with open(feature_dict_path, 'rb') as file: # Open the file in read binary mode
    data = pickle.load(file) # Load the data from the pickle file

In [6]:
# Store test accuracies for each model
test_accuracies_dict = {} # Initialize an empty dictionary to store test accuracies

# Iterate through each model's features
for model_name, values in data.items(): # Loop through each model and its corresponding features
    # Skip MedImageInsightExtractor
    if model_name == "MedImageInsightExtractor": # Skip the MedImageInsightExtractor model
        continue

    # Extract paths and labels for train, val, and test sets
    train_labels = [v["row"]["survival"] for v in values["train"]] # Extract survival labels for the training set
    val_labels = [v["row"]["survival"] for v in values["val"]] # Extract survival labels for the validation set
    test_labels = [v["row"]["survival"] for v in values["test"]] # Extract survival labels for the test set
    
    # Stack features
    train_items = np.vstack([v["feature"] for v in values["train"]]) # Stack features for the training set
    val_items = np.vstack([v["feature"] for v in values["val"]]) # Stack features for the validation set
    test_items = np.vstack([v["feature"] for v in values["test"]]) # Stack features for the test set


    all_items = np.vstack([train_items, val_items, test_items]) # Stack features for all sets
    all_labels = train_labels + val_labels + test_labels # Combine labels for all sets

    # Average across multiple shuffle splits
    n_splits = 5
    split_scores = []
    
    for split in range(n_splits):
        # Get stratified indices for new train/val split with different random seeds
        train_items, train_labels, val_items, val_labels, test_items, test_labels = split_shuffle_data(
            all_items, all_labels, train_ratio=0.5, val_ratio=0.1, random_seed=10+split
        )

        # Train model with hyperparameter optimization
        best_model, study = train_knn_classifier(train_items, train_labels, val_items, val_labels)
        
        # Get score for this split
        split_score = evaluate_model(best_model, test_items, test_labels)
        split_scores.append(split_score)
    
    # Average the scores across splits
    avg_score = np.mean(split_scores)
    
    # Evaluate on test set
    test_accuracies_dict[model_name] = avg_score

[I 2025-02-25 17:11:37,328] A new study created in memory with name: no-name-6cdc723d-7cd7-4d94-8794-24fe07ef55cb
[I 2025-02-25 17:11:37,335] Trial 0 finished with value: 0.4399038461538462 and parameters: {'k': 29}. Best is trial 0 with value: 0.4399038461538462.
[I 2025-02-25 17:11:37,343] Trial 1 finished with value: 0.5324519230769231 and parameters: {'k': 12}. Best is trial 1 with value: 0.5324519230769231.
[I 2025-02-25 17:11:37,350] Trial 2 finished with value: 0.5096153846153846 and parameters: {'k': 11}. Best is trial 1 with value: 0.5324519230769231.
[I 2025-02-25 17:11:37,358] Trial 3 finished with value: 0.39663461538461536 and parameters: {'k': 42}. Best is trial 1 with value: 0.5324519230769231.


[I 2025-02-25 17:11:37,363] Trial 4 finished with value: 0.5673076923076923 and parameters: {'k': 3}. Best is trial 4 with value: 0.5673076923076923.
[I 2025-02-25 17:11:37,369] Trial 5 finished with value: 0.46033653846153844 and parameters: {'k': 28}. Best is trial 4 with value: 0.5673076923076923.
[I 2025-02-25 17:11:37,376] Trial 6 finished with value: 0.3930288461538462 and parameters: {'k': 39}. Best is trial 4 with value: 0.5673076923076923.
[I 2025-02-25 17:11:37,382] Trial 7 finished with value: 0.4411057692307693 and parameters: {'k': 32}. Best is trial 4 with value: 0.5673076923076923.
[I 2025-02-25 17:11:37,389] Trial 8 finished with value: 0.44350961538461536 and parameters: {'k': 23}. Best is trial 4 with value: 0.5673076923076923.
[I 2025-02-25 17:11:37,395] Trial 9 finished with value: 0.5793269230769231 and parameters: {'k': 5}. Best is trial 9 with value: 0.5793269230769231.
[I 2025-02-25 17:11:37,402] Trial 10 finished with value: 0.4254807692307693 and parameters: {

In [7]:
# Plot test accuracies
fig = plot_model_comparison(test_accuracies_dict)
fig.show() # Show the plot

In [8]:
model_features = extract_model_features(data)
model_neighbors = compute_knn_indices(model_features, num_neighbors=10, metric="cosine")
overlap_matrix, model_list = compute_overlap_matrix(model_neighbors)
fig = plot_overlap_matrix(overlap_matrix, model_list)
fig.show()
